In [1]:
!pip install snowflake-snowpark-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.7/419.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 4.2.0
    Uninstalling platformdirs-4.2.0:
      Successfully uninstalled platformdirs-4.2.0


In [2]:
from snowflake.snowpark import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark.functions import year, month, to_date, lit
from snowflake.snowpark.functions import col, avg, stddev_pop, corr
import numpy as np
import pandas as pd
from snowflake.snowpark.exceptions import SnowparkSQLException
import statsmodels.api as sm

In [3]:
parameters = {
...    "account": "YEZEPEO-DATASCIENCE",
...    "user": "VNSHARED",
...    "password": "Columbia2024!",
...    "role": "SYSADMIN", # optional
...    "warehouse": "ANALYSIS", # optional
...   "database": "MEIXI_DATA", # optional
...   "schema":"PUBLIC",  # optional
...  }
session= Session.builder.configs(parameters).create()

Chilean Total GDP, Current Prices, Billion USD (full)

In [ ]:
df_v=session.sql('select * from CONSOLIDATED_COPY ')
df_v = df_v.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_v.columns])
df_v = df_v.select("FECHAEMISION", "DIASCOMPENSACION")

In [ ]:
# Step 1: Extract year and month, then identify unique year-month combinations
df_v_with_year_month = df_v.with_column("YEAR", year(col("FECHAEMISION")))\
                          .with_column("MONTH", month(col("FECHAEMISION")))
unique_year_months = df_v_with_year_month.select("YEAR", "MONTH").distinct().collect()

# Step 2: Iterate and filter for each year-month combination
filtered_dfs = []
for ym_row in unique_year_months:
    year_value, month_value = ym_row["YEAR"], ym_row["MONTH"]
    df_filtered = df_v_with_year_month.filter((col("YEAR") == year_value) & (col("MONTH") == month_value))
    filtered_dfs.append(df_filtered)

# Step 3: Union results
final_df = filtered_dfs[0]
for df in filtered_dfs[1:]:
    final_df = final_df.union(df)

In [ ]:
df_gdp = session.sql('select * from GDP_COPY')
df_gdp = df_gdp.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_gdp.columns])


In [ ]:
# df_gdp.to_pandas()

,EXTRACTIONDATE,INDICATOR,YEAR,DATA
0,2024-03-22,"GDP, current prices (Billions of U.S. dollars)",1980-01-01,29.032
1,2024-03-22,"GDP, current prices (Purchasing power parity; ...",1980-01-01,38.704
2,2024-03-22,"GDP per capita, current prices (Purchasing pow...",1980-01-01,3463.729
3,2024-03-22,Real GDP growth (Annual percent change),1980-01-01,7.900
4,2024-03-22,"GDP, current prices (Billions of U.S. dollars)",1981-01-01,34.374
...,...,...,...,...
191,2024-03-22,Real GDP growth (Annual percent change),2027-01-01,2.400
192,2024-03-22,"GDP, current prices (Billions of U.S. dollars)",2028-01-01,426.681
193,2024-03-22,"GDP, current prices (Purchasing power parity; ...",2028-01-01,735.761
194,2024-03-22,"GDP per capita, current prices (Purchasing pow...",2028-01-01,35821.472


In [ ]:
print(df_gdp.columns)

['EXTRACTIONDATE', 'INDICATOR', 'YEAR', 'DATA']


In [ ]:
df_gdp = df_gdp[df_gdp['INDICATOR'] == 'GDP, current prices (Billions of U.S. dollars)']

In [ ]:
# Extract year and month for 'FechaEmision' in final_df
final_df = final_df.withColumn('Year', year('FECHAEMISION'))
final_df = final_df.withColumn('Month', month('FECHAEMISION'))

# Extract year and month for 'Date' in df_gdp
df_gdp = df_gdp.withColumn('Year1', year('YEAR'))
#df_gdp = df_gdp.withColumn('Month', month('Date (#)'))

# Join the dataframes on year and month
df_with_gdp = final_df.join(df_gdp, (final_df['Year'] == df_gdp['Year1']))

df_with_gdp = df_with_gdp.select(final_df['*'],df_gdp['DATA'])


In [ ]:
# Upload as table to perform linear regression.

# Define the name of the table you want to create or overwrite in Snowflake
table_name = 'table_with_gdp'

# Write the DataFrame to a new table in Snowflake
df_with_gdp.write.mode('overwrite').save_as_table(table_name)

In [ ]:
tb=session.sql('select * from TABLE_WITH_GDP ')
print(tb.columns)

['FECHAEMISION', 'DIASCOMPENSACION', '"l_s7gb_YEAR"', 'MONTH', 'DATA']


In [ ]:
# Perform the linear regression
regression_query = """
SELECT
    REGR_SLOPE("DIASCOMPENSACION", "DATA") AS slope,
    REGR_INTERCEPT("DIASCOMPENSACION", "DATA") AS intercept,
    REGR_R2("DIASCOMPENSACION", "DATA") AS r_squared,
    REGR_COUNT("DIASCOMPENSACION", "DATA") AS sample_size,
    REGR_SXX("DIASCOMPENSACION", "DATA") AS sxx,
    REGR_SYY("DIASCOMPENSACION", "DATA") AS syy,
    REGR_SXY("DIASCOMPENSACION", "DATA") AS sxy
FROM
    table_with_gdp;
"""

# Execute the query
try:
    regression_results = session.sql(regression_query).collect()
    # Print the results
    for result in regression_results:
        print(f"Slope: {result['SLOPE']}")
        print(f"Intercept: {result['INTERCEPT']}")
        print(f"R-squared: {result['R_SQUARED']}")
        print(f"Sample size: {result['SAMPLE_SIZE']}")
        print(f"SXX: {result['SXX']}")
        print(f"SYY: {result['SYY']}")
        print(f"SXY: {result['SXY']}")
except SnowparkSQLException as e:
    print(f"An error occurred: {e}")


Slope: 0.38348491519704275
Intercept: 634.4157962672608
R-squared: 0.0007001341056586646
Sample size: 1812853
SXX: 4015448457.1141205
SYY: 843430675027.8723
SXY: 1539863911.0545044


Chilean Total GDP (LIMITED)

limited to 10000000 rows

In [ ]:
df_v=session.sql('select * from CONSOLIDATED_COPY')
df_v = df_v.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_v.columns])
df_v = df_v.select("FECHAEMISION", "DIASCOMPENSACION")
df_gdp = session.sql('select * from GDP_COPY')
df_gdp = df_gdp.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_gdp.columns])
df_gdp = df_gdp[df_gdp['INDICATOR'] == 'GDP, current prices (Billions of U.S. dollars)']

In [ ]:
# Step 1: Identify unique years
df_v_with_year = df_v.with_column("YEAR", year(col("FECHAEMISION")))
unique_years = df_v_with_year.select("YEAR").distinct().collect()

# Step 2: Iterate and filter for each year
filtered_dfs = []
for year_row in unique_years:
    year_value = year_row["YEAR"]
    df_filtered = df_v_with_year.filter(col("YEAR") == year_value).limit(10000000)
    filtered_dfs.append(df_filtered)

# Step 3: Union results
final_df = filtered_dfs[0]
for df in filtered_dfs[1:]:
    final_df = final_df.union(df)

In [ ]:
df_gdp = df_gdp.to_pandas()
final_df=final_df.to_pandas()

In [ ]:
print(df_gdp.columns)
df_gdp.head(5)

Index(['EXTRACTIONDATE', 'INDICATOR', 'YEAR', 'DATA', 'Date (#)'], dtype='object')


,EXTRACTIONDATE,INDICATOR,YEAR,DATA,Date (#)
0,2024-03-22,"GDP, current prices (Billions of U.S. dollars)",1980-01-01,29.032,1980-01-01
1,2024-03-22,"GDP, current prices (Billions of U.S. dollars)",1981-01-01,34.374,1981-01-01
2,2024-03-22,"GDP, current prices (Billions of U.S. dollars)",1982-01-01,25.630,1982-01-01
3,2024-03-22,"GDP, current prices (Billions of U.S. dollars)",1983-01-01,20.819,1983-01-01
4,2024-03-22,"GDP, current prices (Billions of U.S. dollars)",1984-01-01,20.246,1984-01-01
5,2024-03-22,"GDP, current prices (Billions of U.S. dollars)",1985-01-01,17.360,1985-01-01
6,2024-03-22,"GDP, current prices (Billions of U.S. dollars)",1986-01-01,18.662,1986-01-01
7,2024-03-22,"GDP, current prices (Billions of U.S. dollars)",1987-01-01,22.010,1987-01-01
8,2024-03-22,"GDP, current prices (Billions of U.S. dollars)",1988-01-01,25.947,1988-01-01
9,2024-03-22,"GDP, current prices (Billions of U.S. dollars)",1989-01-01,29.889,1989-01-01


In [ ]:
# Convert 'FechaEmision' to datetime
final_df['FECHAEMISION'] = pd.to_datetime(final_df['FECHAEMISION'])

# 'YEAR' in df_gdp is already in year format, no need to convert to datetime
# Directly use it as an integer
df_gdp['YEAR'] = df_gdp['YEAR'].astype(int)

# Extract the year from 'FechaEmision'
final_df['YEAR'] = final_df['FECHAEMISION'].dt.year

# Merge the dataframes on the 'YEAR' column
# Note: Use pd.merge, not pd.merge([])
df_with_gdp = pd.merge(final_df, df_gdp[['YEAR', 'DATA']], on='YEAR', how='inner')

# Drop the extra 'YEAR' column if you don't need it
df_with_gdp.drop('YEAR', axis=1, inplace=True)

# Check the result
df_with_gdp


,FECHAEMISION,DIASCOMPENSACION,DATA
0,2007-03-31,5.0,172.565
1,2007-04-03,918.0,172.565
2,2007-04-13,79.0,172.565
3,2007-04-23,-7.0,172.565
4,2007-04-27,0.0,172.565
...,...,...,...
1584633,2020-10-08,915.0,254.276
1584634,2020-09-15,262.0,254.276
1584635,2020-06-12,342.0,254.276
1584636,2020-04-14,1210.0,254.276


In [ ]:
# Explanatory model: do not use train test because not predictive

# Convert to numeric and handle non-numeric values
df_with_gdp['DATA'] = pd.to_numeric(df_with_gdp['DATA'], errors='coerce')
df_with_gdp['DIASCOMPENSACION'] = pd.to_numeric(df_with_gdp['DIASCOMPENSACION'], errors='coerce')

# Drop NaN values for a clean analysis
df_with_gdp.dropna(subset=['DATA', 'DIASCOMPENSACION'], inplace=True)

# Prepare the data for modeling
X = df_with_gdp['DIASCOMPENSACION']
Y = df_with_gdp['DATA']
X = sm.add_constant(X)  # adding a constant

# Perform linear regression
model = sm.OLS(Y, X).fit()

# Print the statistics
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   DATA   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                  0.001123
Date:                Tue, 26 Mar 2024   Prob (F-statistic):              0.973
Time:                        12:32:18   Log-Likelihood:            -8.3823e+06
No. Observations:             1582577   AIC:                         1.676e+07
Df Residuals:                 1582575   BIC:                         1.676e+07
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const              243.8415      0.055  

In [ ]:
print("Model Coefficients:")
print(f"Intercept: {model.params[0]:.3f}")
print(f"Slope (DIASCOMPENSACION): {model.params[1]:.3f}")

print("\nModel P-values:")
print(f"P-value (Intercept): {model.pvalues[0]:.3f}")
print(f"P-value (DIASCOMPENSACION): {model.pvalues[1]:.3f}")

print("\nModel R-squared: {:.3f}".format(model.rsquared))
print("Model Standard Errors:")
print(f"Standard Error (Intercept): {model.bse[0]:.3f}")
print(f"Standard Error (DIASCOMPENSACION): {model.bse[1]:.3f}")

Model Coefficients:
Intercept: 243.842
Slope (DIASCOMPENSACION): -0.000

Model P-values:
P-value (Intercept): 0.000
P-value (DIASCOMPENSACION): 0.973

Model R-squared: 0.000
Model Standard Errors:
Standard Error (Intercept): 0.055
Standard Error (DIASCOMPENSACION): 0.000


Santiago GDP Correlation (All Sectors) (full)

Yearly

In [4]:
df_v=session.sql('select * from CONSOLIDATED_COPY ')
df_v = df_v.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_v.columns])
df_v = df_v.select("FECHAEMISION", "DIASCOMPENSACION")

In [5]:
# Step 1: Extract year and month, then identify unique year-month combinations
df_v_with_year_month = df_v.with_column("YEAR", year(col("FECHAEMISION")))\
                          .with_column("MONTH", month(col("FECHAEMISION")))
unique_year_months = df_v_with_year_month.select("YEAR", "MONTH").distinct().collect()

# Step 2: Iterate and filter for each year-month combination
filtered_dfs = []
for ym_row in unique_year_months:
    year_value, month_value = ym_row["YEAR"], ym_row["MONTH"]
    df_filtered = df_v_with_year_month.filter((col("YEAR") == year_value) & (col("MONTH") == month_value))
    filtered_dfs.append(df_filtered)

# Step 3: Union results
final_df = filtered_dfs[0]
for df in filtered_dfs[1:]:
    final_df = final_df.union(df)

In [6]:
df_gdp_s = session.sql('select * from GDP_S_COPY')
df_gdp_s = df_gdp_s.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_gdp_s.columns])


In [7]:
df_gdp_s.to_pandas()

,EXTRACTIONDATE,REG,Descripción series,YEAR,DATE,GDP(million_pesos)
0,2024-03-23,1.0,Agropecuario-silvícola,2013.0,2013-12-31,471.214334
1,2024-03-23,2.0,Pesca,2013.0,2013-12-31,0.000000
2,2024-03-23,3.0,Minería,2013.0,2013-12-31,816.681176
3,2024-03-23,4.0,Industria manufacturera,2013.0,2013-12-31,7067.405752
4,2024-03-23,5.0,"Electricidad, gas, agua y gestión de desechos",2013.0,2013-12-31,1030.944073
...,...,...,...,...,...,...
135,2024-03-23,10.0,Servicios financieros y empresariales,2022.0,2022-12-31,20981.072943
136,2024-03-23,11.0,Servicios de vivienda e inmobiliarios,2022.0,2022-12-31,10695.399499
137,2024-03-23,12.0,Servicios personales,2022.0,2022-12-31,15044.337054
138,2024-03-23,13.0,Administración pública,2022.0,2022-12-31,4968.684958


In [8]:
print(df_gdp_s.columns)

['EXTRACTIONDATE', 'REG', '"Descripción series"', 'YEAR', 'DATE', '"GDP(million_pesos)"']


In [9]:
df_gdp_s = df_gdp_s[df_gdp_s['"Descripción series"'] == 'Producto Interno Bruto']

In [13]:
# Extract year and month for 'FechaEmision' in final_df
final_df = final_df.withColumn('Year', year('FECHAEMISION'))
final_df = final_df.withColumn('Month', month('FECHAEMISION'))

# Extract year and month for 'Date' in df_gdp_s
df_gdp_s = df_gdp_s.withColumn('Year1', year('DATE'))
#df_gdp = df_gdp.withColumn('Month', month('Date (#)'))

# Join the dataframes on year and month
df_with_gdp_s = final_df.join(df_gdp_s, (final_df['Year'] == df_gdp_s['Year1']))

df_with_gdp_s = df_with_gdp_s.select(final_df['*'],df_gdp_s['"GDP(million_pesos)"'])


In [14]:
# Upload as table to perform linear regression.
# Define the name of the table you want to create or overwrite in Snowflake
table_name = 'table_with_gdp_s'

# Write the DataFrame to a new table in Snowflake
df_with_gdp_s.write.mode('overwrite').save_as_table(table_name)

In [15]:
tb=session.sql('select * from TABLE_WITH_GDP_S ')
print(tb.columns)

['FECHAEMISION', 'DIASCOMPENSACION', '"l_rh6p_YEAR"', 'MONTH', '"GDP(million_pesos)"']


In [19]:
regression_query = """
SELECT
    REGR_SLOPE("DIASCOMPENSACION", "GDP(million_pesos)") AS slope,
    REGR_INTERCEPT("DIASCOMPENSACION", "GDP(million_pesos)") AS intercept,
    REGR_R2("DIASCOMPENSACION", "GDP(million_pesos)") AS r_squared,
    REGR_COUNT("DIASCOMPENSACION", "GDP(million_pesos)") AS sample_size,
    REGR_SXX("DIASCOMPENSACION", "GDP(million_pesos)") AS sxx,
    REGR_SYY("DIASCOMPENSACION", "GDP(million_pesos)") AS syy,
    REGR_SXY("DIASCOMPENSACION", "GDP(million_pesos)") AS sxy
FROM
    table_with_gdp_s;
"""
# Execute the query
try:
    regression_results = session.sql(regression_query).collect()
    # Print the results
    for result in regression_results:
        print(f"Slope: {result['SLOPE']}")
        print(f"Intercept: {result['INTERCEPT']}")
        print(f"R-squared: {result['R_SQUARED']}")
        print(f"Sample size: {result['SAMPLE_SIZE']}")
        print(f"SXX: {result['SXX']}")
        print(f"SYY: {result['SYY']}")
        print(f"SXY: {result['SXY']}")
except SnowparkSQLException as e:
    print(f"An error occurred: {e}")


Slope: -0.007506282623956027
Intercept: 1346.7928220951376
R-squared: 0.01370552378204466
Sample size: 1113367
SXX: 120986636860877.0
SYY: 497383749098.9662
SXY: -908159889999.6797


Santiago GDP Correlation (All Sectors) (limited)

10000000 rows

In [20]:
df_v=session.sql('select * from CONSOLIDATED_COPY')
df_v = df_v.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_v.columns])
df_v = df_v.select("FECHAEMISION", "DIASCOMPENSACION")
df_gdp = session.sql('select * from GDP_S_COPY')
df_gdp_s = df_gdp_s.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_gdp_s.columns])
df_gdp_s = df_gdp_s[df_gdp_s['"Descripción series"'] == 'Producto Interno Bruto']

In [21]:
# Step 1: Identify unique years
df_v_with_year = df_v.with_column("YEAR", year(col("FECHAEMISION")))
unique_years = df_v_with_year.select("YEAR").distinct().collect()

# Step 2: Iterate and filter for each year
filtered_dfs = []
for year_row in unique_years:
    year_value = year_row["YEAR"]
    df_filtered = df_v_with_year.filter(col("YEAR") == year_value).limit(10000000)
    filtered_dfs.append(df_filtered)

# Step 3: Union results
final_df = filtered_dfs[0]
for df in filtered_dfs[1:]:
    final_df = final_df.union(df)

In [22]:
df_gdp_s = df_gdp_s.to_pandas()
final_df=final_df.to_pandas()

In [24]:
print(df_gdp_s.columns)
df_gdp_s.head(5)

Index(['EXTRACTIONDATE', 'REG', 'Descripción series', 'YEAR', 'DATE',
       'GDP(million_pesos)', 'YEAR1'],
      dtype='object')


,EXTRACTIONDATE,REG,Descripción series,YEAR,DATE,GDP(million_pesos),YEAR1
0,2024-03-23,14.0,Producto Interno Bruto,2013.0,2013-12-31,57950.692393,2013
1,2024-03-23,14.0,Producto Interno Bruto,2014.0,2014-12-31,61747.513018,2014
2,2024-03-23,14.0,Producto Interno Bruto,2015.0,2015-12-31,67395.594568,2015
3,2024-03-23,14.0,Producto Interno Bruto,2016.0,2016-12-31,72096.318083,2016
4,2024-03-23,14.0,Producto Interno Bruto,2017.0,2017-12-31,75041.892228,2017


In [26]:
# Convert 'FechaEmision' to datetime
final_df['FECHAEMISION'] = pd.to_datetime(final_df['FECHAEMISION'])

# 'YEAR1' in df_gdp_s is already in year format, no need to convert to datetime
# Directly use it as an integer
df_gdp_s['YEAR'] = df_gdp_s['YEAR'].astype(int)

# Extract the year from 'FechaEmision'
final_df['YEAR'] = final_df['FECHAEMISION'].dt.year

# Merge the dataframes on the 'YEAR' column
# Note: Use pd.merge, not pd.merge([])
df_with_gdp_s = pd.merge(final_df, df_gdp_s[['YEAR', 'GDP(million_pesos)']], on='YEAR', how='inner')

# Drop the extra 'YEAR' column if you don't need it
df_with_gdp_s.drop('YEAR', axis=1, inplace=True)

# Check the result
df_with_gdp_s


,FECHAEMISION,DIASCOMPENSACION,GDP(million_pesos)
0,2015-08-18,8.0,67395.594568
1,2015-08-21,-7.0,67395.594568
2,2015-08-28,23.0,67395.594568
3,2015-08-28,10.0,67395.594568
4,2015-09-04,-16.0,67395.594568
...,...,...,...
913684,2020-12-14,689.0,79602.544458
913685,2020-06-08,636.0,79602.544458
913686,2020-11-03,736.0,79602.544458
913687,2020-06-12,709.0,79602.544458


In [28]:
# Explanatory model: do not use train test because not predictive

# Convert to numeric and handle non-numeric values
df_with_gdp_s['GDP(million_pesos)'] = pd.to_numeric(df_with_gdp_s['GDP(million_pesos)'], errors='coerce')
df_with_gdp_s['DIASCOMPENSACION'] = pd.to_numeric(df_with_gdp_s['DIASCOMPENSACION'], errors='coerce')

# Drop NaN values for a clean analysis
df_with_gdp_s.dropna(subset=['GDP(million_pesos)', 'DIASCOMPENSACION'], inplace=True)

X = df_with_gdp_s['DIASCOMPENSACION']
Y = df_with_gdp_s['GDP(million_pesos)']

# Adding a constant to X
X = sm.add_constant(X)

# Perform linear regression
model = sm.OLS(Y, X).fit()

# Print the statistics
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:     GDP(million_pesos)   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                 1.079e+04
Date:                Wed, 27 Mar 2024   Prob (F-statistic):               0.00
Time:                        08:35:23   Log-Likelihood:            -9.7527e+06
No. Observations:              912294   AIC:                         1.951e+07
Df Residuals:                  912292   BIC:                         1.951e+07
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const              7.69e+04     16.682  

In [29]:
print("Model Coefficients:")
print(f"Intercept: {model.params[0]:.3f}")
print(f"Slope (DIASCOMPENSACION): {model.params[1]:.3f}")

print("\nModel P-values:")
print(f"P-value (Intercept): {model.pvalues[0]:.3f}")
print(f"P-value (DIASCOMPENSACION): {model.pvalues[1]:.3f}")

print("\nModel R-squared: {:.3f}".format(model.rsquared))
print("Model Standard Errors:")
print(f"Standard Error (Intercept): {model.bse[0]:.3f}")
print(f"Standard Error (DIASCOMPENSACION): {model.bse[1]:.3f}")

Model Coefficients:
Intercept: 76897.848
Slope (DIASCOMPENSACION): -1.824

Model P-values:
P-value (Intercept): 0.000
P-value (DIASCOMPENSACION): 0.000

Model R-squared: 0.012
Model Standard Errors:
Standard Error (Intercept): 16.682
Standard Error (DIASCOMPENSACION): 0.018
